In [ ]:
# 必要なライブラリをインストール
!pip install fastapi uvicorn nest-asyncio discord.py python-dotenv requests

# 必要なライブラリのインポート
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import threading
import nest_asyncio
import discord
from discord import app_commands
import requests

# Colab用の非同期ループ設定
nest_asyncio.apply()

# FastAPIアプリの作成
app = FastAPI()

class InputNumber(BaseModel):
    value: int

@app.post("/api/return_number")
async def return_number(input: InputNumber):
    return {"received": input.value}

@app.get("/api/hello")
def read_root():
    return {"message": "Hello from FastAPI!"}

# FastAPIサーバーをバックグラウンドで起動
def run_fastapi():
    uvicorn.run(app, host="127.0.0.1", port=8000)

threading.Thread(target=run_fastapi).start()
print("FastAPIサーバーが起動しました...")

# Discord Botの設定
class MyBot(discord.Client):
    def __init__(self):
        super().__init__(intents=discord.Intents.default())
        self.tree = app_commands.CommandTree(self)

    async def setup_hook(self):
        # コマンド同期
        await self.tree.sync()
        print("スラッシュコマンドが同期されました！")

bot = MyBot()

# FastAPIローカルホストURL
FASTAPI_URL = "http://127.0.0.1:8000"

# スラッシュコマンド: FastAPIのPOSTエンドポイントを叩く
@bot.tree.command(name="send_number", description="数値をFastAPIに送信します")
async def send_number(interaction: discord.Interaction, value: int):
    try:
        response = requests.post(f"{FASTAPI_URL}/api/return_number", json={"value": value})
        data = response.json()
        await interaction.response.send_message(f"FastAPIからのレスポンス: {data['received']}")
    except Exception as e:
        await interaction.response.send_message(f"エラーが発生しました: {e}")

# スラッシュコマンド: FastAPIのGETエンドポイントを叩く
@bot.tree.command(name="hello", description="FastAPIのhelloエンドポイントを呼び出します")
async def hello(interaction: discord.Interaction):
    try:
        response = requests.get(f"{FASTAPI_URL}/api/hello")
        data = response.json()
        await interaction.response.send_message(data["message"])
    except Exception as e:
        await interaction.response.send_message(f"エラーが発生しました: {e}")

# Bot起動
print("Discord Botを起動します...")
bot.run("DISCOR_BOT_TOKEN")
